# Preprocessing Data

## To maintain quality we have Drop Rows.
- Year Column having some alpha value and they are object we have to convert it to int.
- Price Column having some alpha value and they are object we have to convert it to int and remove commas.
- Kms_driven Column having some alpha value and they are object we have to convert it to int and remove commas and kms at the end.
- Fuel having nan data we have to remove.
- Keep first three words of name


In [1]:
import pandas as pd

In [2]:
# Reading CSV
carData=pd.read_csv("quikr_car.csv")

In [3]:
backup=carData.copy()

## Cleaning

Cleaning Year Columns

In [4]:
carData=carData[carData['year'].str.isnumeric()]
carData['year']=carData['year'].astype(int)

Clenaing Price Column

In [5]:
carData=carData[carData['Price']!="Ask For Price"]
carData['Price']=carData['Price'].str.replace(',','').astype(int)

Cleaning Kms_Driven Data

In [6]:
carData['kms_driven']=carData['kms_driven'].str.replace(',','')
carData['kms_driven']=carData['kms_driven'].str.replace(' kms','')
carData=carData[carData['kms_driven'].str.isnumeric()]
carData['kms_driven']=carData['kms_driven'].astype(int)

Clenaing Fuel Type

In [7]:
carData=carData[~carData['fuel_type'].isna()]

Cleaning Name

In [8]:
carData['name']=carData['name'].str.split(' ').str.slice(0,3).str.join(' ')

In [9]:
carData.reset_index(drop=True)

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
2,Hyundai Grand i10,Hyundai,2014,325000,28000,Petrol
3,Ford EcoSport Titanium,Ford,2014,575000,36000,Diesel
4,Ford Figo,Ford,2012,175000,41000,Diesel
...,...,...,...,...,...,...
811,Maruti Suzuki Ritz,Maruti,2011,270000,50000,Petrol
812,Tata Indica V2,Tata,2009,110000,30000,Diesel
813,Toyota Corolla Altis,Toyota,2009,300000,132000,Petrol
814,Tata Zest XM,Tata,2018,260000,27000,Diesel


# Now Analyzing preprocessed Data to find lies

In [10]:
carData.describe()

,year,Price,kms_driven
count,816.000000,8.160000e+02,816.000000
mean,2012.444853,4.117176e+05,46275.531863
std,4.002992,4.751844e+05,34297.428044
min,1995.000000,3.000000e+04,0.000000
25%,2010.000000,1.750000e+05,27000.000000
50%,2013.000000,2.999990e+05,41000.000000
75%,2015.000000,4.912500e+05,56818.500000
max,2019.000000,8.500003e+06,400000.000000


In [11]:
carData[carData['Price']>6e6] #There is only  one car which is more than 60 lakhs so this data should be removed

,name,company,year,Price,kms_driven,fuel_type
562,Mahindra XUV500 W6,Mahindra,2014,8500003,45000,Diesel


In [12]:
carData=carData[carData['Price']<=6e6] 

In [13]:
carData.reset_index(drop=True)

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
2,Hyundai Grand i10,Hyundai,2014,325000,28000,Petrol
3,Ford EcoSport Titanium,Ford,2014,575000,36000,Diesel
4,Ford Figo,Ford,2012,175000,41000,Diesel
...,...,...,...,...,...,...
810,Maruti Suzuki Ritz,Maruti,2011,270000,50000,Petrol
811,Tata Indica V2,Tata,2009,110000,30000,Diesel
812,Toyota Corolla Altis,Toyota,2009,300000,132000,Petrol
813,Tata Zest XM,Tata,2018,260000,27000,Diesel


In [14]:
carData.to_csv("cleanCarData.csv")

# Model

In this on x axis we have all data then price and on y axis we have data of price

In [15]:
x=carData.drop(columns='Price')
y=carData['Price']

# Spilting into test and train set
We have split data into test and train with ratio of 4:1

In [31]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_Test=train_test_split(x,y,test_size=0.2,random_state=7314)

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

# OneHotEncoder
We use one hot encoder for classifying the Category by numbers because machine learning is all about mathematical equation.It only understand numeric value.

In [32]:
ohe=OneHotEncoder()
ohe.fit(x[['name','company','fuel_type']])

OneHotEncoder()

In [33]:
colums_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['name','company','fuel_type']),remainder="passthrough")

In [34]:
colums_trans

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder',
                                 OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat Diesel',
       'Chevrolet Beat LS', 'Chevrolet B...
       'Volkswagen Vento Konekt', 'Volvo S80 Summum'], dtype=object),
                                                           array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                           array(['Diesel', 'LPG', 'Petrol'], dtype=object)]),
                                 ['name', 'company', 'fuel_type'])])

In [35]:
lr=LinearRegression()

In [36]:
pipe=make_pipeline(colums_trans,lr)

In [37]:
pipe.fit(X_train,Y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat...
                                                                            array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                                            array(['Diesel', 'LPG', 'Petrol'], dtype=object)]),
                                                  ['name', 'company',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [38]:
y_predict=pipe.predict(X_test)

# Calculating Accuracy

In [39]:
r2_score(y_predict,Y_Test)*100

87.41546464609924

# This stage is done to find max r2 score at ith random_state

In [28]:
scores=[]
for i in range(10000):
    X_train,X_test,Y_train,Y_Test=train_test_split(x,y,test_size=0.2,random_state=i)
    lr=LinearRegression()
    pipe=make_pipeline(colums_trans,lr)
    pipe.fit(X_train,Y_train)
    y_predict=pipe.predict(X_test)
    scores.append(r2_score(y_predict,Y_Test)*100)

In [43]:
max(scores)

87.41546464609924

We Reached at 87.41 % of accuracy

# Saving Model

In [44]:
import pickle

In [46]:
pickle.dump(pipe,open("CarPricePredictionModel.pkl","wb"))

# Prediction

In [61]:
data=['Hyundai Santro Xing','Hyundai',2020,45000,'Diesel']
columns=['name','company','year','kms_driven','fuel_type']
pipe.predict(pd.DataFrame([data],columns=columns))

array([673031.16824959])

In [62]:
carData['fuel_type'].unique()

array(['Petrol', 'Diesel', 'LPG'], dtype=object)

In [63]:
carData['company'].unique()

array(['Hyundai', 'Mahindra', 'Ford', 'Maruti', 'Skoda', 'Audi', 'Toyota',
       'Renault', 'Honda', 'Datsun', 'Mitsubishi', 'Tata', 'Volkswagen',
       'Chevrolet', 'Mini', 'BMW', 'Nissan', 'Hindustan', 'Fiat', 'Force',
       'Mercedes', 'Land', 'Jaguar', 'Jeep', 'Volvo'], dtype=object)